In [32]:
from bs4 import BeautifulSoup
import urllib.request as urllib2
import re
import csv
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

Schwab_BASE = 'https://www.schwabjobs.com'
PNC_BASE = 'https://www.indeed.com/viewjob?'

PNC_URL = "https://www.indeed.com/jobs?q=Pnc+Bank"
Schwab_URL = "https://www.schwabjobs.com/List/Custom/Schwab-Job-Category"

stateList = []

bank = 'PNC Bank'
listId = 1
jobNo = 1
BASE = 'https://www.indeed.com/'
FIRST_LISTING_PAGE = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?clearPrefilter=1'
LISTING_URL_PREFIX = 'https://www.indeed.com/cmp/Pnc-Financial-Services-Group/jobs?start='
LISTING_URL_POSTFIX = 'clearPrefilter=1#cmp-menu-container'

def keywordsFileName(listId):
    if listId == 1:
        return 'frequency.csv'
    elif listId == 2:
        return 'textRank.csv'
    else:
        return 'tfIdfWords.csv'

def scrape(postingUrl): 
    print("posting - " + postingUrl)
    global bank
    global listId
    global jobNo
    
    headers = ["Job No", "Institution", "List Id {1,2,3}", "URL (URL of the job posting)", "List id (1,2,3)"]
    values = [jobNo, bank, listId, postingUrl, listId]
    
    indices = list(range(1, 101))
    
    wordCountDict = dict.fromkeys(indices,0)
    postingDict = dict(zip(headers, values))
    
    enc = 'utf-8'
    with open(keywordsFileName(listId), 'r', encoding = enc) as f:
        reader = csv.reader(f)
        keywords = list(reader)
    arr = [i[0] for i in keywords]
    
    try:
        hdr = {'User-Agent': 'Mozilla/5.0'}
        request = urllib2.Request(postingUrl,headers=hdr)
        html_page = urlopen(request)
        soup2 =  BeautifulSoup(html_page,"html.parser")
        if bank == 'PNC Bank':
            value = soup2.find('input', {'id':'preLoadJSON'})
            for word in value.get('value').split('"CandLandPageText"')[1].replace("\\u003", " ").split():
                if(word in arr):
                    ind = arr.index(word)
                    wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
            dataDict = postingDict.copy()  
            return {**postingDict, **wordCountDict}
                
        else:
            value = soup2.findAll("div", {"class": "desc"})
            for val in value:
                st = val.text.split()
                for word in st:
                    if(word in arr):
                        ind = arr.index(word)
                        wordCountDict[ind] = wordCountDict.get(ind, 0) + 1
            jobNo = jobNo + 1
    except:
        return ("An exception occurred")

    dataDict = postingDict.copy()  
    return {**postingDict, **wordCountDict} 



def scrape_pages(pages):
    stateList = {}
    for p in pages:
        pageHtml = urlopen(p)
        soup = BeautifulSoup(pageHtml)
        table = soup.find('table', 'JobListTable')
        pageDataframe = pd.DataFrame([])
        for link in table.findAll('a', attrs={'href': re.compile("/ShowJob")}):
            page = Schwab_BASE + link.get('href')
            state = table.findAll("td",{"class": "colstate"})
            for s in state:
                st = re.sub(r'[^ \w\.]', '', s.text)
                if st not in stateList:
                    stateList[st] = 1
                else:
                    stateList[st] += 1
            pageDict = scrape(page)
            if (type(pageDict) is dict):
                data = pd.DataFrame([pageDict])
                # if file does not exist write header 
                if not os.path.isfile('schwaData.csv'):
                    data.to_csv('schwaData.csv', header='column_names')
                else: # else it exists so append without writing the header
                    data.to_csv('schwaData.csv', mode='a', header=False)
    with open('states.csv', 'a+', encoding = 'utf-8') as f: 
        for key in stateList.keys():
            f.write("%s,%d\n"%(key,stateList[key]))
        

def get_pages(categories):
    pageLinks = []
    for key, value in categories.items():
        if("---" not in key):
            if(value > 30):
                pageCount = 1
                for i in range(1,value,29):
                    pageLinks.append(Schwab_BASE + key + "Page-" + str(pageCount))
                    pageCount = pageCount + 1
                
            else:
                pageLinks.append(Schwab_BASE + key)
    return pageLinks

def get_category(response):
    bs_obj = BeautifulSoup(response, "html.parser")
    if bs_obj:
        cat_table = bs_obj.find_all("table",{"class": "list-categores-table"})

        catLinks = []
        numPages = []
        cat = []

        for page in cat_table:
            linkTags = page.find_all('a', attrs={'href': re.compile("/ListJobs")})
            divTags = page.find_all('div')
            for tag in linkTags:
                catLinks.append(tag.get('href'))
                cat.append(tag.text)
            for div in divTags:
                num = int(re.search(r'\d+', div.text).group())
                numPages.append(num)
        #categCsv = dict(zip(cat, numPages))
        #plt.bar(range(len(categCsv)), list(categCsv.values()), align='center')
        #plt.xticks(range(len(categCsv)), list(categCsv.keys()))
        #plt.imshow()
        return dict(zip(catLinks, numPages))

def scrape_site(Url):
    scrape_page(FIRST_LISTING_PAGE)
    for i in range(12,996,12):
        nextPageUrl = LISTING_URL_PREFIX +  str(i) + "&" + LISTING_URL_POSTFIX
        print(nextPageUrl)
        scrape_page(nextPageUrl)
    global count
    print(count)

def get_data (Url, bank):
    if (bank == "PNC BANK"):
        scrape_site(Url)
    elif (bank == "Schwab Bank"):
        res = urllib2.urlopen("https://www.schwabjobs.com/List/Custom/Schwab-Job-Category")
        categories = get_category(res)
        pages = get_pages(categories)
        scrape_pages(pages)

if __name__ == '__main__':
    get_data(Schwab_URL , "Schwab Bank")
    #get_data(PNC_URL , "PNC Bank")


posting - https://www.schwabjobs.com/ShowJob/Id/1949474/Manager,-Finance/
posting - https://www.schwabjobs.com/ShowJob/Id/1947943/Deal-Desk-Sourcing-Manager/
posting - https://www.schwabjobs.com/ShowJob/Id/1939618/Deal-Desk-Sourcing-Manager/
posting - https://www.schwabjobs.com/ShowJob/Id/1840064/Senior-Team-Manager,-Technology-Procurement,-Vendor-Sourcing-Management/
posting - https://www.schwabjobs.com/ShowJob/Id/1941909/Managing-Director,-CRE-Finance/
posting - https://www.schwabjobs.com/ShowJob/Id/1863067/Sr.-Manager,-Finance/
posting - https://www.schwabjobs.com/ShowJob/Id/1919260/Sr.-Manager,-Treasury/
posting - https://www.schwabjobs.com/ShowJob/Id/1677626/Senior-Team-Manager,-Finance/
posting - https://www.schwabjobs.com/ShowJob/Id/1931170/Senior-Finance-Specialist/
posting - https://www.schwabjobs.com/ShowJob/Id/1932656/Senior-Team-Manager,-Finance/
posting - https://www.schwabjobs.com/ShowJob/Id/1878137/Manager,-Finance/
posting - https://www.schwabjobs.com/ShowJob/Id/1905839

posting - https://www.schwabjobs.com/ShowJob/Id/1876809/Advisor-Services-Manager/
posting - https://www.schwabjobs.com/ShowJob/Id/1822268/Resolution-Associate,-Partner-Support/
posting - https://www.schwabjobs.com/ShowJob/Id/1942847/Client-Relationship-Specialist-Denver,-CO/
posting - https://www.schwabjobs.com/ShowJob/Id/1942676/Legal-Counsel-Digital,-Technology-Transactions-Cybersecurity/
posting - https://www.schwabjobs.com/ShowJob/Id/1885227/Sr.-Client-Relationship-Specialist-Stamford,-CT/
posting - https://www.schwabjobs.com/ShowJob/Id/1941986/Client-Relationship-Specialist-West-Hartford,-CT/
posting - https://www.schwabjobs.com/ShowJob/Id/1941984/Client-Relationship-Specialist-Harrisburg,-PA/
posting - https://www.schwabjobs.com/ShowJob/Id/1827171/Service-Relationship-Manager-Advisor-Family-Office/
posting - https://www.schwabjobs.com/ShowJob/Id/1864404/Financial-Service-Professional/
posting - https://www.schwabjobs.com/ShowJob/Id/1807289/Point-of-Contact-Advice-Professional-Sch

posting - https://www.schwabjobs.com/ShowJob/Id/1883821/VP,-Financial-Consultant-Naperville-Geneva/
posting - https://www.schwabjobs.com/ShowJob/Id/1861817/VP,-Financial-Consultant/
posting - https://www.schwabjobs.com/ShowJob/Id/1866266/Associate-Financial-Consultant-St-Paul/
posting - https://www.schwabjobs.com/ShowJob/Id/1911520/VP,-Financial-Consultant-Bridgewater,-NJ/
posting - https://www.schwabjobs.com/ShowJob/Id/1911096/Vice-President,-Financial-Consultant-Omaha,-NE/
posting - https://www.schwabjobs.com/ShowJob/Id/1777713/VP,-Financial-Consultant-Fremont,-CA/
posting - https://www.schwabjobs.com/ShowJob/Id/1778758/Vice-President,-Financial-Consultant-Cupertino,-CA/
posting - https://www.schwabjobs.com/ShowJob/Id/1909297/VP,-Financial-Consultant-Novi,-MI/
posting - https://www.schwabjobs.com/ShowJob/Id/1941985/Associate-Financial-Consultant-Morristown,-NJ/
posting - https://www.schwabjobs.com/ShowJob/Id/1908554/Associate-Financial-Consultant-Woodland-Hills,-CA/
posting - https:/

posting - https://www.schwabjobs.com/ShowJob/Id/1946023/Business-System-Analyst-#CET/
posting - https://www.schwabjobs.com/ShowJob/Id/1945932/Sr.-Site-Reliability-Engineer-DevOps-#CET/
posting - https://www.schwabjobs.com/ShowJob/Id/1945923/Software-Application-Engineer-C#-.NET-#-CET/
posting - https://www.schwabjobs.com/ShowJob/Id/1945801/Sr-Manager-New-Employment-Recruitment-and-Development/
posting - https://www.schwabjobs.com/ShowJob/Id/1945775/Senior-Manager-New-Employment-Recruitment-and-Development/
posting - https://www.schwabjobs.com/ShowJob/Id/1725584/Staff,-Java-Developer/
posting - https://www.schwabjobs.com/ShowJob/Id/1827164/Application-Integration-Engineer/
posting - https://www.schwabjobs.com/ShowJob/Id/1633511/Sr.-Staff-DB2-DBA/
posting - https://www.schwabjobs.com/ShowJob/Id/1911799/Managing-Director-of-Security-Design-(ERN4)/
posting - https://www.schwabjobs.com/ShowJob/Id/1851211/Managing-Director,-Software-Applications-Engineer/
posting - https://www.schwabjobs.com

posting - https://www.schwabjobs.com/ShowJob/Id/1927244/Sr.-Manager-Software-Applications-Engineering/
posting - https://www.schwabjobs.com/ShowJob/Id/1736810/Senior-C#-Developer/
posting - https://www.schwabjobs.com/ShowJob/Id/1745374/Technical-Director/
posting - https://www.schwabjobs.com/ShowJob/Id/1932080/Sr.-Software-Development-Engineer-in-Test/
posting - https://www.schwabjobs.com/ShowJob/Id/1937897/Senior-Staff-Engineer-Senior-.Net-Developer/
posting - https://www.schwabjobs.com/ShowJob/Id/1934386/Technical-Director,-Program-Project-Manager/
posting - https://www.schwabjobs.com/ShowJob/Id/1797065/Sr.-Site-Reliability-DevOps-Engineer/
posting - https://www.schwabjobs.com/ShowJob/Id/1937111/Data-Center-Systems-Engineer/
posting - https://www.schwabjobs.com/ShowJob/Id/1931556/Staff-CI-CD-Release-Engineer/
posting - https://www.schwabjobs.com/ShowJob/Id/1733683/Software-Application-Engineer/
posting - https://www.schwabjobs.com/ShowJob/Id/1920564/Release-Automation-Engineer/
posti

posting - https://www.schwabjobs.com/ShowJob/Id/1827175/Senior-Security-Application-Pen-Tester-(ERN4)/
posting - https://www.schwabjobs.com/ShowJob/Id/1902970/Staff-C#-.Net-Developer/
posting - https://www.schwabjobs.com/ShowJob/Id/1902623/Staff-DevOps-Engineer/
posting - https://www.schwabjobs.com/ShowJob/Id/1904052/Associate-Software-Engineer-New-Employee-Recruitment-and-Development-(NERD)/
posting - https://www.schwabjobs.com/ShowJob/Id/1867324/Senior-Salesforce-Developer-Administrator/
posting - https://www.schwabjobs.com/ShowJob/Id/1901258/Sr.-Business-Systems-Analyst/
posting - https://www.schwabjobs.com/ShowJob/Id/1904218/Staff-TRAC-Engagement-Audit-Liaison-(ERN4)/
posting - https://www.schwabjobs.com/ShowJob/Id/1460868/Big-Data-Solution-Architect/
posting - https://www.schwabjobs.com/ShowJob/Id/1897726/Sr.-Software-Engineer/
posting - https://www.schwabjobs.com/ShowJob/Id/1704384/Full-Stack-.NET-Developer-Investments/
posting - https://www.schwabjobs.com/ShowJob/Id/1745894/Agil

posting - https://www.schwabjobs.com/ShowJob/Id/1941924/Sr.-Manager,-Client-Experience/
posting - https://www.schwabjobs.com/ShowJob/Id/1892985/Product-Owner-Business-Analyst/
posting - https://www.schwabjobs.com/ShowJob/Id/1811109/Senior-Program-Manager,-Advisor-Services-Advisor-Family-Office/
posting - https://www.schwabjobs.com/ShowJob/Id/1884727/Sr-Manager-Security-Management/
posting - https://www.schwabjobs.com/ShowJob/Id/1939461/Program-Coordinator/
posting - https://www.schwabjobs.com/ShowJob/Id/1926215/Retirement-Operations-Specialist/
posting - https://www.schwabjobs.com/ShowJob/Id/1945076/Sr.-Business-Acceptance-Process-Consultant/
posting - https://www.schwabjobs.com/ShowJob/Id/1889101/Conversion-Analyst/
posting - https://www.schwabjobs.com/ShowJob/Id/1945073/Communications-Manager/
posting - https://www.schwabjobs.com/ShowJob/Id/1631355/Large-Mega-Market-Relationship-Manager,-Field-Vice-President,-Retirement-Plan-Services/
posting - https://www.schwabjobs.com/ShowJob/Id/1

posting - https://www.schwabjobs.com/ShowJob/Id/1908553/Stock-Plan-Services-Financial-Services-Professional/
posting - https://www.schwabjobs.com/ShowJob/Id/1905288/Financial-Reporting-Analyst-Stock-Plan-Services/
posting - https://www.schwabjobs.com/ShowJob/Id/1901080/Managing-Director,-SPS-Strategy-and-Participant-Experience/
posting - https://www.schwabjobs.com/ShowJob/Id/1898513/Stock-Plan-Services-Client-Service-Administrator/
posting - https://www.schwabjobs.com/ShowJob/Id/1898524/Stock-Plan-Services-Client-Service-Administrator/


In [ ]:
! pip install matplotlib